What will cover here:-
1. Will Run hyper-Parametermeter tuning while training a modal
2. Will log every hyper-parameter and metic using MLFow UI
3. Compare the various Runs results in MLFlow UI
4. Choose the best Run and register it in MLFlow registry

In [20]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing


In [21]:
housing = fetch_california_housing()
housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

Prepairing the DataSet

In [22]:
data = pd.DataFrame(data=housing.data,columns=housing.feature_names)
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [23]:
data['Actual_price'] = housing.target
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Actual_price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


Train Test Split, Model HyperParameter Tuning MLFLow Experiments

In [24]:
from urllib.parse import urlparse 
# This is library that will be used for MLFlow experiments
from mlflow.models import infer_signature

In [25]:
X = data.drop(columns=["Actual_price"])
Y = data["Actual_price"]

In [26]:
# Split Data in Traing and test 
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.20)

In [27]:
signature = infer_signature(X_train,Y_train)

In [28]:
# Defining Hyper-Parameter Grid
param_grid = {
    'n_estimators':[100,200],
    'max_depth': [5,10,None],
    'min_samples_split': [2,5],
    'min_samples_leaf':[1,2]
}
# These are the some of hyper-parameters used in RandomForest


In [29]:
# Hyper-Parameter Tunning using Grid Dearchcv
def hyperParameter_tuning(X_train,Y_train,param_grid):
    rf = RandomForestRegressor()
    grid_search = GridSearchCV(estimator=rf,param_grid=param_grid,
                               cv=3, n_jobs=-1,verbose=2,
                               scoring='neg_mean_squared_error')
    grid_search.fit(X_train,Y_train)
    return grid_search

In [34]:
# Start the MLFlow Experimets

with mlflow.start_run():
    # Perform Hyper-Parameter tuning
    grid_search = hyperParameter_tuning(X_train,Y_train,param_grid)
    
    # Get the best Model
    best_model = grid_search.best_estimator_

    # Evaluate the best model
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(Y_test,y_pred)
    
    # Log best Parameters and metrics
    mlflow.log_param("best_n_estimators",grid_search.best_params_['n_estimators'])
    mlflow.log_param("best_max_depth",grid_search.best_params_['max_depth'])
    mlflow.log_param("best_min_samples_split",grid_search.best_params_['min_samples_split'])
    mlflow.log_param("best_min_samples_leaf",grid_search.best_params_['min_samples_leaf'])

    # Log the metrics
    mlflow.log_metric("mse",mse)

    # Tracking URL
    mlflow.set_tracking_uri(uri='http://127.0.0.1:5000')
    tracking_url_store = urlparse(mlflow.get_tracking_uri()).scheme

    if tracking_url_store != 'file':
        mlflow.sklearn.log_model(best_model,'model',
                                 registered_model_name='Best Random Forest Model')
    else:
        mlflow.sklearn.log_model(best_model,"model",
                                 signature=signature)

    print(f"Best Hyper-Parameters:{grid_search.best_params_}")
    print(f"mean_squared_error:{mse}")
 



Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   5.1s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   5.1s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   5.1s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   5.2s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   5.5s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   9.4s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   4.6s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=  10.3s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=  10.5s
[CV] END max_depth=5, min_samples_leaf=

2024/11/04 17:59:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'Best Random Forest Model' already exists. Creating a new version of this model...
2024/11/04 17:59:54 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Best Random Forest Model, version 2
Created version '2' of model 'Best Random Forest Model'.
2024/11/04 17:59:54 INFO mlflow.tracking._tracking_service.client: 🏃 View run magnificent-newt-120 at: http://127.0.0.1:5000/#/experiments/0/runs/be73e3f6ca06420f965db6d679f8a121.
2024/11/04 17:59:54 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0.


Best Hyper-Parameters:{'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
mean_squared_error:0.2649517099405718
